# Geoguessr Image Analysis
## Mason Mines: Up-and-Coming Geoguessr Fan

**Introduction:** Geoguessr is a game in which players are placed in a random google-street view somewhere across the globe, and are tasked with identifying where they are based on their surroundings. Points are awarded based on the distance between the guessed and actual location (ranging from 0-5000 points, with 5000 representing essentially adjacent guesses).

**Purpose:** By analyzing street-view data, I hope to gain a greater understanding of global locations and their common attributes. At the very least, I hope to gain more information about the locations chosen during games to gain a competitive edge.

In [9]:
import pandas as pd

data = pd.read_csv('data/coords.csv')

# Display the first few rows of the data
print(data.head())

# Optional: Check basic information about the data
print(data.info())



FileNotFoundError: [Errno 2] No such file or directory: 'data/coords.csv'

### Initial Data Analysis: [xxx]

## Initial cleaning


In [13]:
# Remove


# Find the most frequent values in a column
most_frequent = data['Airline Name'].value_counts()

# Display the most frequent values
print(most_frequent)


Airline Name
Aegean Airlines      100
Vueling Airlines     100
PLAY                 100
Volotea              100
Norwegian            100
                    ... 
TUIfly Nordic          1
Air Labrador           1
Air Costa              1
Air Cote d'Ivoire      1
Voyage Air             1
Name: count, Length: 497, dtype: int64



## First approach: Statistic based

Different approaches
- Ranked based on whatever
-
